In [9]:
# Install required packages.
import os
import torch

!pip install pyg-lib -f https://data.pyg.org/whl/torch-2.3.0+cpu.html # PyG for working with graphs
# !pip install git+https://github.com/pyg-team/pytorch_geometric.git # more PyG
# !pip install pytorch_frame[full] #PyTorch Frame for working with tabular data
# !pip install relbench[full]

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [1]:
# Install required packages.
import os
import torch

In [2]:
import torch
print(torch.__version__)

2.3.0


In [3]:
import relbench

relbench.__version__

'1.1.0'

In [4]:
import numpy as np

from torch.nn import BCEWithLogitsLoss, L1Loss
from relbench.datasets import get_dataset
from relbench.tasks import get_task

dataset = get_dataset("rel-hm", download=True)
task = get_task("rel-hm", "user-churn", download=True)

train_table = task.get_table("train")
val_table = task.get_table("val")
test_table = task.get_table("test")

out_channels = 1
loss_fn = L1Loss()
tune_metric = "roc_auc"
higher_is_better = True

100%|########################################| 143M/143M [00:00<00:00, 139GB/s]
Unzipping contents of 'C:\Users\KN2C\AppData\Local\relbench\relbench\Cache\rel-hm\db.zip' to 'C:\Users\KN2C\AppData\Local\relbench\relbench\Cache\rel-hm\.'
100%|#####################################| 12.1M/12.1M [00:00<00:00, 12.1GB/s]
Unzipping contents of 'C:\Users\KN2C\AppData\Local\relbench\relbench\Cache\rel-hm\tasks\user-churn.zip' to 'C:\Users\KN2C\AppData\Local\relbench\relbench\Cache\rel-hm\tasks\.'


Let's check out the training table just to make sure it looks fine.

In [5]:
train_table

Table(df=
         timestamp  customer_id  churn
0       2020-06-08      1255921      0
1       2020-06-08      1256165      0
2       2020-06-08      1256239      1
3       2020-06-08      1256569      1
4       2020-06-08      1256721      1
...            ...          ...    ...
3832687 2019-12-09      1100160      1
3832688 2019-12-09      1100373      1
3832689 2019-12-09      1100716      1
3832690 2019-12-09      1101037      1
3832691 2019-12-09      1101199      1

[3832692 rows x 3 columns],
  fkey_col_to_pkey_table={'customer_id': 'customer'},
  pkey_col=None,
  time_col=timestamp)

Note that to load the data we did not require any deep learning libraries. Now we introduce the PyTorch Frame library, which is useful for encoding individual tables into initial node features.

In [6]:
import os
import math
import numpy as np
from tqdm import tqdm

import torch
import torch_geometric
import torch_frame

# Some book keeping
from torch_geometric.seed import seed_everything

seed_everything(42)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)  # check that it's cuda if you want it to run in reasonable time!
root_dir = "./data"

c:\Users\KN2C\miniconda3\envs\kamal\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


The first big move is to build a graph out of the database. Here we use our pre-prepared conversion function.

The source code can be found at: https://github.com/snap-stanford/relbench/blob/main/relbench/modeling/graph.py

Each node in the graph corresonds to a single row in the database. Crucially, PyTorch Frame stores whole tables as objects in a way that is compatibile with PyG minibatch sampling, meaning we can sample subgraphs as in https://arxiv.org/abs/1706.02216, and retrieve the relevant raw features.

PyTorch Frame also stores the `stype` (i.e., modality) of each column, and any specialized feature encoders (e.g., text encoders) to be used later. So we need to configure the `stype` for each column, for which we use a function that tries to automatically detect the `stype`.

In [7]:
from relbench.modeling.utils import get_stype_proposal

db = dataset.get_db()
col_to_stype_dict = get_stype_proposal(db)
col_to_stype_dict

Loading Database object from C:\Users\KN2C\AppData\Local\relbench\relbench\Cache/rel-hm/db...
Done in 1.17 seconds.


{'article': {'article_id': <stype.numerical: 'numerical'>,
  'product_code': <stype.numerical: 'numerical'>,
  'prod_name': <stype.text_embedded: 'text_embedded'>,
  'product_type_no': <stype.numerical: 'numerical'>,
  'product_type_name': <stype.text_embedded: 'text_embedded'>,
  'product_group_name': <stype.text_embedded: 'text_embedded'>,
  'graphical_appearance_no': <stype.numerical: 'numerical'>,
  'graphical_appearance_name': <stype.text_embedded: 'text_embedded'>,
  'colour_group_code': <stype.numerical: 'numerical'>,
  'colour_group_name': <stype.text_embedded: 'text_embedded'>,
  'perceived_colour_value_id': <stype.numerical: 'numerical'>,
  'perceived_colour_value_name': <stype.text_embedded: 'text_embedded'>,
  'perceived_colour_master_id': <stype.numerical: 'numerical'>,
  'perceived_colour_master_name': <stype.text_embedded: 'text_embedded'>,
  'department_no': <stype.numerical: 'numerical'>,
  'department_name': <stype.text_embedded: 'text_embedded'>,
  'index_code': <sty

If trying a new dataset, you should definitely check through this dict of `stype`s to check that look right, and manually change any mistakes by the auto-detection function.

Next we also define our text encoding model, which we use GloVe embeddings for speed and convenience. Feel free to try alternatives here.

In [8]:
# !pip install -U sentence-transformers # we need another package for text encoding
from typing import List, Optional
from sentence_transformers import SentenceTransformer
from torch import Tensor


class GloveTextEmbedding:
    def __init__(self, device: Optional[torch.device
                                       ] = None):
        self.model = SentenceTransformer(
            "sentence-transformers/average_word_embeddings_glove.6B.300d",
            device=device,
        )

    def __call__(self, sentences: List[str]) -> Tensor:
        return torch.from_numpy(self.model.encode(sentences))



In [9]:
from torch_frame.config.text_embedder import TextEmbedderConfig
from relbench.modeling.graph import make_pkey_fkey_graph

text_embedder_cfg = TextEmbedderConfig(
    text_embedder=GloveTextEmbedding(device=device), batch_size=256
)

data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,  # speficied column types
    text_embedder_cfg=text_embedder_cfg,  # our chosen text encoder
    cache_dir=os.path.join(
        root_dir, f"rel-hm_materialized_cache"
    ),  # store materialized graph for convenience
)

Embedding raw data in mini-batch: 100%|██████████| 5360/5360 [00:33<00:00, 162.40it/s]


We can now check out `data`, our main graph object. `data` is a heterogeneous and temporal graph, with node types given by the table it originates from.

In [10]:
data

HeteroData(
  article={ tf=TensorFrame([105542, 24]) },
  customer={ tf=TensorFrame([1371980, 6]) },
  transactions={
    tf=TensorFrame([15187287, 3]),
    time=[15187287],
  },
  (transactions, f2p_customer_id, customer)={ edge_index=[2, 15187287] },
  (customer, rev_f2p_customer_id, transactions)={ edge_index=[2, 15187287] },
  (transactions, f2p_article_id, article)={ edge_index=[2, 15187287] },
  (article, rev_f2p_article_id, transactions)={ edge_index=[2, 15187287] }
)

We can also check out the TensorFrame for one table like this:

In [11]:
data["customer"].tf

TensorFrame(
  num_cols=6,
  num_rows=1371980,
  categorical (3): ['Active', 'FN', 'fashion_news_frequency'],
  numerical (1): ['age'],
  embedding (2): ['club_member_status', 'postal_code'],
  has_target=False,
  device='cpu',
)

This may be a little confusing at first, as in graph ML it is more standard to associate to the graph object `data` a tensor, e.g., `data.x` for which `data.x[idx]` is a 1D array/tensor storing all the features for node with index `idx`.

But actually this `data` object behaves similarly. For a given node type, e.g., `races` again, `data['races']` stores two pieces of information


In [12]:
list(data["customer"].keys())

['tf']

A `TensorFrame` object, and a timestamp for each node. The `TensorFrame` object acts analogously to the usual tensor of node features, and you can simply use indexing to retrieve the features of a single row (node), or group of nodes.

In [13]:
data["customer"].tf[10]

TensorFrame(
  num_cols=6,
  num_rows=1,
  categorical (3): ['Active', 'FN', 'fashion_news_frequency'],
  numerical (1): ['age'],
  embedding (2): ['club_member_status', 'postal_code'],
  has_target=False,
  device='cpu',
)

In [14]:
data["customer"].tf[10:20]

TensorFrame(
  num_cols=6,
  num_rows=10,
  categorical (3): ['Active', 'FN', 'fashion_news_frequency'],
  numerical (1): ['age'],
  embedding (2): ['club_member_status', 'postal_code'],
  has_target=False,
  device='cpu',
)

We can also check the edge indices between two different node types, such as `races` amd `circuits`. Note that the edges are also heterogenous, so we also need to specify which edge type we want to look at. Here we look at `f2p_curcuitId`, which are the directed edges pointing _from_ a race (the `f` stands for `foreign key`), _to_ the circuit at which te race happened (the `p` stands for `primary key`).

In [15]:
data[("transactions", "f2p_customer_id", "customer")]

{'edge_index': tensor([[       0,        1,        2,  ..., 15187284, 15187285, 15187286],
        [     155,      155,      155,  ...,  1371926,  1371926,  1371926]])}

Now we are ready to instantiate our data loaders. For this we will need to import PyTorch Geometric, our GNN library. Whilst we're at it let's add a seed.


In [16]:
from relbench.modeling.graph import get_node_train_table_input, make_pkey_fkey_graph
from torch_geometric.loader import NeighborLoader

loader_dict = {}

for split, table in [
    ("train", train_table),
    ("val", val_table),
    ("test", test_table),
]:
    table_input = get_node_train_table_input(
        table=table,
        task=task,
    )
    entity_table = table_input.nodes[0]
    loader_dict[split] = NeighborLoader(
        data,
        num_neighbors=[
            128 for i in range(2)
        ],  # we sample subgraphs of depth 2, 128 neighbors per node.
        time_attr="time",
        input_nodes=table_input.nodes,
        input_time=table_input.time,
        transform=table_input.transform,
        batch_size=512,
        temporal_strategy="uniform",
        shuffle=split == "train",
        num_workers=0,
        persistent_workers=False,
    )

Now we need our model...




In [17]:
from torch.nn import BCEWithLogitsLoss
import copy
from typing import Any, Dict, List

import torch
from torch import Tensor
from torch.nn import Embedding, ModuleDict
from torch_frame.data.stats import StatType
from torch_geometric.data import HeteroData
from torch_geometric.nn import MLP
from torch_geometric.typing import NodeType

from relbench.modeling.nn import HeteroEncoder, HeteroGraphSAGE, HeteroTemporalEncoder


class Model(torch.nn.Module):

    def __init__(
        self,
        data: HeteroData,
        col_stats_dict: Dict[str, Dict[str, Dict[StatType, Any]]],
        num_layers: int,
        channels: int,
        out_channels: int,
        aggr: str,
        norm: str,
        # List of node types to add shallow embeddings to input
        shallow_list: List[NodeType] = [],
        # ID awareness
        id_awareness: bool = False,
    ):
        super().__init__()

        self.encoder = HeteroEncoder(
            channels=channels,
            node_to_col_names_dict={
                node_type: data[node_type].tf.col_names_dict
                for node_type in data.node_types
            },
            node_to_col_stats=col_stats_dict,
        )
        self.temporal_encoder = HeteroTemporalEncoder(
            node_types=[
                node_type for node_type in data.node_types if "time" in data[node_type]
            ],
            channels=channels,
        )
        self.gnn = HeteroGraphSAGE(
            node_types=data.node_types,
            edge_types=data.edge_types,
            channels=channels,
            aggr=aggr,
            num_layers=num_layers,
        )
        self.head = MLP(
            channels,
            out_channels=out_channels,
            norm=norm,
            num_layers=1,
        )
        self.embedding_dict = ModuleDict(
            {
                node: Embedding(data.num_nodes_dict[node], channels)
                for node in shallow_list
            }
        )

        self.id_awareness_emb = None
        if id_awareness:
            self.id_awareness_emb = torch.nn.Embedding(1, channels)
        self.reset_parameters()

    def reset_parameters(self):
        self.encoder.reset_parameters()
        self.temporal_encoder.reset_parameters()
        self.gnn.reset_parameters()
        self.head.reset_parameters()
        for embedding in self.embedding_dict.values():
            torch.nn.init.normal_(embedding.weight, std=0.1)
        if self.id_awareness_emb is not None:
            self.id_awareness_emb.reset_parameters()

    def forward(
        self,
        batch: HeteroData,
        entity_table: NodeType,
    ) -> Tensor:
        seed_time = batch[entity_table].seed_time
        x_dict = self.encoder(batch.tf_dict)

        rel_time_dict = self.temporal_encoder(
            seed_time, batch.time_dict, batch.batch_dict
        )

        for node_type, rel_time in rel_time_dict.items():
            x_dict[node_type] = x_dict[node_type] + rel_time

        for node_type, embedding in self.embedding_dict.items():
            x_dict[node_type] = x_dict[node_type] + embedding(batch[node_type].n_id)

        x_dict = self.gnn(
            x_dict,
            batch.edge_index_dict,
            batch.num_sampled_nodes_dict,
            batch.num_sampled_edges_dict,
        )

        return self.head(x_dict[entity_table][: seed_time.size(0)])

    def forward_dst_readout(
        self,
        batch: HeteroData,
        entity_table: NodeType,
        dst_table: NodeType,
    ) -> Tensor:
        if self.id_awareness_emb is None:
            raise RuntimeError(
                "id_awareness must be set True to use forward_dst_readout"
            )
        seed_time = batch[entity_table].seed_time
        x_dict = self.encoder(batch.tf_dict)
        # Add ID-awareness to the root node
        x_dict[entity_table][: seed_time.size(0)] += self.id_awareness_emb.weight

        rel_time_dict = self.temporal_encoder(
            seed_time, batch.time_dict, batch.batch_dict
        )

        for node_type, rel_time in rel_time_dict.items():
            x_dict[node_type] = x_dict[node_type] + rel_time

        for node_type, embedding in self.embedding_dict.items():
            x_dict[node_type] = x_dict[node_type] + embedding(batch[node_type].n_id)

        x_dict = self.gnn(
            x_dict,
            batch.edge_index_dict,
        )

        return self.head(x_dict[dst_table])


model = Model(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)


# if you try out different RelBench tasks you will need to change these
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
epochs = 10

We also need standard train/test loops

In [18]:
def train() -> float:
    model.train()

    loss_accum = count_accum = 0
    for batch in tqdm(loader_dict["train"]):
        batch = batch.to(device)

        optimizer.zero_grad()
        pred = model(
            batch,
            task.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred

        loss = loss_fn(pred.float(), batch[entity_table].y.float())
        loss.backward()
        optimizer.step()

        loss_accum += loss.detach().item() * pred.size(0)
        count_accum += pred.size(0)

    return loss_accum / count_accum


@torch.no_grad()
def test(loader: NeighborLoader) -> np.ndarray:
    model.eval()

    pred_list = []
    for batch in loader:
        batch = batch.to(device)
        pred = model(
            batch,
            task.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        pred_list.append(pred.detach().cpu())
    return torch.cat(pred_list, dim=0).numpy()

Now we are ready to train!

In [19]:
state_dict = None
best_val_metric = -math.inf if higher_is_better else math.inf
for epoch in range(1, epochs + 1):
    train_loss = train()
    val_pred = test(loader_dict["val"])
    val_metrics = task.evaluate(val_pred, val_table)
    print(f"Epoch: {epoch:02d}, Train loss: {train_loss}, Val metrics: {val_metrics}")

    if (higher_is_better and val_metrics[tune_metric] > best_val_metric) or (
        not higher_is_better and val_metrics[tune_metric] < best_val_metric
    ):
        best_val_metric = val_metrics[tune_metric]
        state_dict = copy.deepcopy(model.state_dict())


model.load_state_dict(state_dict)
val_pred = test(loader_dict["val"])
val_metrics = task.evaluate(val_pred, val_table)
print(f"Best Val metrics: {val_metrics}")

test_pred = test(loader_dict["test"])
test_metrics = task.evaluate(test_pred)
print(f"Best test metrics: {test_metrics}")

100%|██████████| 7486/7486 [11:03<00:00, 11.29it/s]


Epoch: 01, Train loss: 0.1895827813143176, Val metrics: {'average_precision': np.float64(0.8923428345832963), 'accuracy': 0.8128036992528346, 'f1': np.float64(0.8967365849792118), 'roc_auc': np.float64(0.6835030040348632)}


100%|██████████| 7486/7486 [11:12<00:00, 11.13it/s]


Epoch: 02, Train loss: 0.18504184390924702, Val metrics: {'average_precision': np.float64(0.8918181191642063), 'accuracy': 0.8128036992528346, 'f1': np.float64(0.8967365849792118), 'roc_auc': np.float64(0.6824367463857932)}


100%|██████████| 7486/7486 [11:14<00:00, 11.10it/s]


...

Epoch: 08, Train loss: 0.18249780826046738, Val metrics: {'average_precision': np.float64(0.8897208308544237), 'accuracy': 0.8150765452740477, 'f1': np.float64(0.8961586703145974), 'roc_auc': np.float64(0.681861404049845)}


100%|██████████| 7486/7486 [11:15<00:00, 11.08it/s]


Epoch: 09, Train loss: 0.18215467025013346, Val metrics: {'average_precision': np.float64(0.8916205665986776), 'accuracy': 0.815403103610429, 'f1': np.float64(0.8959949366343337), 'roc_auc': np.float64(0.684689230198527)}


100%|██████████| 7486/7486 [11:15<00:00, 11.08it/s]


Epoch: 10, Train loss: 0.18171188158134324, Val metrics: {'average_precision': np.float64(0.8811135774680492), 'accuracy': 0.8166440252886775, 'f1': np.float64(0.8970192286529672), 'roc_auc': np.float64(0.6717641704162609)}
Best Val metrics: {'average_precision': np.float64(0.8926973081852648), 'accuracy': 0.8128036992528346, 'f1': np.float64(0.8967365849792118), 'roc_auc': np.float64(0.6862514208424542)}
Best test metrics: {'average_precision': np.float64(0.8984164554730769), 'accuracy': 0.8261347636607442, 'f1': np.float64(0.9047905774540328), 'roc_auc': np.float64(0.681332186880603)}
